In [1]:
import os
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import plotly.express as px


In [2]:
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')
username = os.getenv('SPOTIPY_USERNAME')

In [3]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)

scope = 'user-library-read playlist-read-private'

try:
    token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + username)

In [4]:
import pandas as pd
import numpy as np

def convert_to_minutes_seconds(ms):
    seconds = ms // 1000
    minutes = int(seconds // 60)
    seconds = seconds % 60
    return f"{minutes}m{seconds}s"

class SpotifyAPI:
    def __init__(self, username, client_id, client_secret, redirect_uri):
        scope = "user-library-read playlist-read-private"
        self.username = username

        try:
            token = util.prompt_for_user_token(username, scope,client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
            self.sp = spotipy.Spotify(auth = token)
        except:
            raise ValueError(f"Token is not accesible for {username}")

    def get_playlist_audio_features(self, playlist_id):
        offset = 0
        songs = []
        items = []
        ids = []
        while True:
            content = self.sp.user_playlist_tracks(self.username, playlist_id, fields=None, limit=100, offset=offset, market=None)
            songs += content["items"]
            if content["next"] is not None:
                offset += 100
            else:
                break

        for i in songs:
            ids.append(i["track"]["id"])

        index = 0
        audio_features = []
        while index < len(ids):
            audio_features += self.sp.audio_features(ids[index:index + 50])
            index += 50

        features_list = []
        for features in audio_features:
            features_list.append([features["energy"], features["liveness"],
                                features["tempo"], features["speechiness"],
                                features["acousticness"], features["instrumentalness"],
                                features["time_signature"], features["danceability"],
                                features["key"], features["duration_ms"],
                                features["loudness"], features["valence"],
                                features["mode"], features["type"],
                                features["uri"]])

        df = pd.DataFrame(features_list, columns=["energy", "liveness",
                                                "tempo", "speechiness",
                                                "acousticness", "instrumentalness",
                                                "time_signature", "danceability",
                                                "key", "duration_ms", "loudness",
                                                "valence", "mode", "type", "uri"])
        df['duration_str'] = df['duration_ms'].apply(convert_to_minutes_seconds)

        def get_track_name(uri):
            return self.sp.track(uri)['name']

        def get_artist_name(uri):
            artist = self.sp.track(uri)['artists']
            names = [artist[x]['name'] for x,_ in enumerate(artist)]
            artists = ', '.join(names)
            return artists

        df['music_name'] = df['uri'].apply(get_track_name)
        df['artist_name'] = df['uri'].apply(get_artist_name)
        df['mode_str'] = np.where(df['mode'] == 1, 'Major', 'Minor')
        df['key_str'] = np.where(df['key'] == 0, 'C', np.where(df['key'] == 1, 'C#/Db',
                                                            np.where(df['key'] == 2, 'D',
                                                                    np.where(df['key'] == 3, 'D#/Eb',
                                                                            np.where(df['key'] == 4, 'E',
                                                                                    np.where(df['key'] == 5, 'F',
                                                                                            np.where(df['key'] == 6, 'F#/Gb',
                                                                                                    np.where(df['key'] == 7, 'G',
                                                                                                            np.where(df['key'] == 8, 'G#/Ab',
                                                                                                                    np.where(df['key'] == 9, 'A',
                                                                                                                            np.where(df['key'] == 10, 'A#/Bb',
                                                                                                                                    np.where(df['key'] == 11, 'B', 'None'))))))))))))

        df = df[[
            'energy',
            'liveness',
            'speechiness',
            'acousticness',
            'instrumentalness',
            'time_signature',
            'danceability',
            'loudness',
            'valence',
            'key',
            "key_str",
            'mode',
            "mode_str",
            'tempo',
            'duration_ms',
            'duration_str',
            'music_name',
            'artist_name',
        ]]
        return df


In [5]:
features = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('0QGaSSt6trrQM3Mo6LUCil')
features

,energy,liveness,speechiness,acousticness,instrumentalness,time_signature,danceability,loudness,valence,key,key_str,mode,mode_str,tempo,duration_ms,duration_str,music_name,artist_name
0,0.620,0.2070,0.2870,0.4890,0.000000,4,0.737,-6.172,0.536,8,G#/Ab,1,Major,111.984,171471,2m51s,Fragrância - Remix (feat. PL Quest),"MD Chefe, L7NNON, Offlei Sounds, PL Quest"
1,0.446,0.1310,0.0477,0.2640,0.000000,4,0.746,-7.967,0.492,8,G#/Ab,1,Major,126.025,152452,2m32s,Tiffany,"MD Chefe, Offlei Sounds"
2,0.318,0.1380,0.3670,0.1280,0.000002,4,0.896,-12.411,0.480,9,A,1,Major,119.964,152052,2m32s,Rei Lacoste,"MD Chefe, DomLaike, Offlei Sounds"
3,0.453,0.1020,0.1770,0.1910,0.005810,4,0.876,-8.578,0.313,1,C#/Db,1,Major,118.043,197030,3m17s,HB20 2 (LQ) - Bonus Track (feat. Flacko & C'97),"MD Chefe, Rare G, Offlei Sounds, Flacko, C'97"
4,0.645,0.0818,0.0609,0.3450,0.000057,4,0.759,-8.365,0.513,1,C#/Db,1,Major,109.916,164182,2m44s,Burberry,"Offlei Sounds, MD Chefe, Kloe"
5,0.382,0.0709,0.3110,0.4150,0.000000,4,0.945,-9.671,0.830,0,C,0,Minor,127.967,136875,2m16s,DAMN!,"Ryu, the Runner, Luk The Real, Emitê Único, 6e..."
6,0.551,0.1260,0.2710,0.0608,0.000000,4,0.866,-9.294,0.658,0,C,1,Major,139.212,138129,2m18s,Pura Adrenalina,"Ryu, the Runner, dvrkness13"
7,0.430,0.1830,0.2490,0.2590,0.000000,4,0.886,-9.155,0.519,8,G#/Ab,1,Major,134.048,163881,2m43s,Mantém o Pique,"Ryu, the Runner, Neckklace, Salve Crazy"


In [6]:
liked_musics = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('751h3UgPuX8Q5WYubUbsdr')

In [7]:
dopamine_flux = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('46UUqBAAPqmLwW9FMg3t3w')
one_music = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('43eJMveAngLgS5zO7rb8jE')
calm_trap = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('0QGaSSt6trrQM3Mo6LUCil')
dopamine_flux_focus = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('7aZtjKLxA2z8cM2tpw522Q')
indie_musics = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('23UP8JocqQArs7iY5bLZJF')
phonk = SpotifyAPI(username, client_id, client_secret, redirect_uri).get_playlist_audio_features('265e9UI8kP6Om5bUvr32Qp')

In [8]:
def get_numeric_columns(df):
    return df[(df.dtypes != 'object')[(df.dtypes != 'object')].index]

linear_features = features[(features.dtypes != 'object')[(features.dtypes != 'object')].index]

liked_musics_numeric = get_numeric_columns(liked_musics.set_index('music_name'))

dopamine_flux_numeric = get_numeric_columns(dopamine_flux.set_index('music_name'))
one_music_numeric = get_numeric_columns(one_music.set_index('music_name'))
calm_trap_numeric = get_numeric_columns(calm_trap.set_index('music_name'))
dopamine_flux_focus_numeric = get_numeric_columns(dopamine_flux_focus.set_index('music_name'))
indie_musics_numeric = get_numeric_columns(indie_musics.set_index('music_name'))
phonk_numeric = get_numeric_columns(phonk.set_index('music_name'))

,energy,liveness,speechiness,acousticness,instrumentalness,time_signature,danceability,loudness,valence,key,mode,tempo,duration_ms,Dopamine
music_name,,,,,,,,,,,,,,
Al Compás De Mi Caballo,0.576,0.0421,0.1460,0.70200,0.000254,4,0.549,-8.757,0.969,11,0,205.522,205485,1
Pollos Hermanos Veneno,0.441,0.3010,0.0724,0.52700,0.017900,4,0.690,-10.345,0.818,9,0,170.048,144240,1
Mi Dulce Sueño,0.488,0.2220,0.2930,0.61200,0.000000,4,0.628,-10.050,0.959,8,1,103.778,286883,1
La Pelea Con el Diablo,0.912,0.0603,0.0566,0.13400,0.001960,4,0.793,-6.772,0.920,2,1,85.125,168516,1
El Cucuy,0.712,0.2440,0.3070,0.56900,0.000000,4,0.599,-5.032,0.960,4,1,184.133,158256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM YUM,0.904,0.0807,0.3430,0.31600,0.023900,4,0.769,-1.545,0.155,1,1,130.059,129697,1
THE AUTOTREM 1.0,0.934,0.1990,0.3120,0.10600,0.000000,4,0.713,0.057,0.527,1,1,129.912,149107,1
YUMMI,0.810,0.1670,0.0497,0.65300,0.472000,4,0.752,-2.684,0.553,5,1,130.109,96279,1


In [18]:
import plotly.express as px

def plot_ratio_diff(last_liked_music, liked_musics):
    f1 = pd.concat([liked_musics.mean().rename('liked_musics'), last_liked_music.mean().rename('last_liked_music')], axis=1)
    f1 = f1.drop(['instrumentalness'])
    f1['diff'] = f1['last_liked_music'] - f1['liked_musics']
    f1['ratio %'] = (f1['last_liked_music'] - f1['liked_musics']) / f1['liked_musics'] * 100
    return px.bar(f1, y=['ratio %'], template='plotly_dark')

def liked_diff(last_liked_music, liked_musics):
    f1 = pd.concat([liked_musics.mean().rename('liked_musics'), last_liked_music.mean().rename('last_liked_music')], axis=1)
    f1 = f1.drop(['instrumentalness'])
    f1['diff'] = f1['last_liked_music'] - f1['liked_musics']
    f1['ratio %'] = (f1['last_liked_music'] - f1['liked_musics']) / f1['liked_musics'] * 100
    return f1

plot_ratio_diff(dopamine_flux_numeric, liked_musics_numeric)

In [17]:
liked_diff(dopamine_flux_focus_numeric, liked_musics_numeric)

,liked_musics,last_liked_music,diff,ratio %
energy,0.663372,0.54900,-0.114372,-17.241033
liveness,0.170879,0.09190,-0.078979,-46.219307
speechiness,0.134035,0.11895,-0.015085,-11.254339
acousticness,0.185113,0.26800,0.082887,44.776429
time_signature,3.975000,4.00000,0.025000,0.628931
danceability,0.765192,0.81750,0.052308,6.835978
loudness,-6.619789,-8.47150,-1.851711,27.972359
valence,0.534038,0.41300,-0.121038,-22.664647
key,5.311111,1.00000,-4.311111,-81.171548
mode,0.577778,1.00000,0.422222,73.076923


Copilot Prompt:

explain the difference between dopamine_flux_focus and liked_musics based on this dataframe:

liked_musics	last_liked_music	diff	ratio %
energy	0.663372	0.54900	-0.114372	-17.241033
liveness	0.170879	0.09190	-0.078979	-46.219307
speechiness	0.134035	0.11895	-0.015085	-11.254339
acousticness	0.185113	0.26800	0.082887	44.776429
time_signature	3.975000	4.00000	0.025000	0.628931
danceability	0.765192	0.81750	0.052308	6.835978
loudness	-6.619789	-8.47150	-1.851711	27.972359
valence	0.534038	0.41300	-0.121038	-22.664647
key	5.311111	1.00000	-4.311111	-81.171548
mode	0.577778	1.00000	0.422222	73.076923
tempo	128.791036	113.97950	-14.811536	-11.500440
duration_ms	196676.758333	180606.00000	-16070.758333	-8.171153



Copilot Answer

Based on the provided dataframe, here are the differences between the dopamine_flux_focus and liked_musics playlists:

1. energy: The liked_musics playlist has 17.24% less energy than the dopamine_flux_focus playlist.

2. liveness: The liked_musics playlist has 46.22% less liveness than the dopamine_flux_focus playlist.

3. speechiness: The liked_musics playlist has 11.25% less speechiness than the dopamine_flux_focus playlist.

4. acousticness: The liked_musics playlist has 44.78% more acousticness than the dopamine_flux_focus playlist.

5. time_signature: The liked_musics playlist has a 0.63% higher time signature than the dopamine_flux_focus playlist.

6. danceability: The liked_musics playlist is 6.84% more danceable than the dopamine_flux_focus playlist.

7. loudness: The liked_musics playlist is 27.97% louder than the dopamine_flux_focus playlist.

8. valence: The liked_musics playlist has 22.66% less valence than the dopamine_flux_focus playlist.

9. key: The liked_musics playlist has 81.17% less key than the dopamine_flux_focus playlist.

10. mode: The liked_musics playlist has 73.08% more mode than the dopamine_flux_focus playlist.

11. tempo: The liked_musics playlist has a 11.50% slower tempo than the dopamine_flux_focus playlist.

12. duration_ms: The liked_musics playlist has tracks that are 8.17% shorter in duration than the dopamine_flux_focus playlist.

These differences suggest that the liked_musics playlist tends to have more acoustic, danceable, and louder tracks with a higher mode and time signature, but less energy, liveness, speechiness, valence, key, tempo, and shorter duration than the dopamine_flux_focus playlist.

